# Importing packages

In [1]:
using JuMP
using GLPK
using Formatting
using GLPKMathProgInterface

# Reading problem instance and defining Constants

In [ ]:
instance_file = "Problema2/smt_50_250.dat"

In [4]:
f = open(instance_file, "r")         
 
lines = readlines(f)
 
close(f)

n, m, s, t = [parse(Int, num_char) for num_char in split(lines[1])]
print("number of vertices (n): $n\n")
print("number of edges (m): $m\n")
print("initial vertice (s): $s\n")
print("final vertice (t): $t\n")

V = Array(1:n)
V_st = copy(V)
deleteat!(V_st, V_st .== s)
deleteat!(V_st, V_st .== t)

D = zeros(Int64, (n, n))
G = zeros(Int64, (n, n))
P = zeros(Int64, (n, n, n))

print("\nEdges and weights: \n")
for i in 2:m+1
    u, v, d = [parse(Int, num_char) for num_char in split(lines[i])]
    print("{$u, $v} = $d\n")
    D[u, v] = d
    D[v, u] = d
    G[u, v] = 1
    G[v, u] = 1
end

for i in 1:n
    for j in 1:n
        for k in 1:n
            P[i,j,k] = abs(D[i,j] - D[j,k])
        end
    end
end

M = maximum(P);

number of vertices (n): 50
number of edges (m): 250
initial vertice (s): 1
final vertice (t): 3

Edges and weights: 
{27, 12} = 7488
{47, 2} = 1318
{49, 25} = 4763
{44, 18} = 1006
{48, 31} = 9296
{42, 38} = 5897
{27, 3} = 8397
{32, 17} = 9265
{16, 11} = 3897
{33, 19} = 3610
{50, 24} = 3898
{47, 33} = 8085
{22, 13} = 4111
{29, 20} = 1887
{19, 7} = 727
{6, 4} = 853
{42, 14} = 5668
{30, 21} = 4313
{36, 1} = 849
{36, 4} = 9178
{47, 31} = 3099
{32, 10} = 4571
{9, 4} = 465
{40, 6} = 4712
{48, 27} = 7458
{48, 29} = 8398
{33, 20} = 2042
{43, 9} = 636
{46, 13} = 6190
{34, 28} = 4822
{36, 31} = 1817
{14, 5} = 2075
{42, 26} = 4175
{18, 17} = 3794
{42, 21} = 9188
{32, 11} = 1829
{25, 8} = 4556
{40, 39} = 2008
{21, 9} = 7768
{17, 3} = 3881
{45, 9} = 6373
{20, 15} = 5446
{25, 15} = 9436
{10, 3} = 8111
{22, 18} = 8211
{4, 3} = 2659
{50, 11} = 7108
{45, 19} = 6643
{31, 27} = 2021
{45, 29} = 1603
{23, 1} = 3555
{2, 1} = 6221
{48, 19} = 6993
{25, 20} = 1431
{48, 40} = 8401
{35, 23} = 4899
{37, 18} = 701

# Defining Model

In [5]:
model = Model(GLPK.Optimizer)

@variable(model, C[1:n, 1:n], Bin)
@variable(model, Q[1:n, 1:n, 1:n], Bin)
@variable(model, p >= 0, Int)

@objective(model, Min, p)

# Restrições para gerar um caminho C subgrafo de G
@constraint(model, sum(C[j,s] for j = 1:n) == 0)
@constraint(model, sum(C[s,j] for j = 1:n) == 1)
@constraint(model, sum(C[j,t] for j = 1:n) == 1)
@constraint(model, sum(C[t,j] for j = 1:n) == 0)
@constraint(model, [i in V_st], sum(C[j,i] for j = 1:n) <= 1)
@constraint(model, [i in V_st], sum(C[j,i] for j = 1:n) - sum(C[i,j] for j = 1:n) == 0)
@constraint(model, [i = 1:n, j=1:n], C[i,j] - G[i,j] <= 0)

# Restrições para que  Q_{ijk} = C_{ij} ^ C_{jk}
@constraint(model, [i = 1:n, j=1:n, k=1:n], Q[i,j,k] <= (C[i,j] + C[j,k])/2)
@constraint(model, [i = 1:n, j=1:n, k=1:n], Q[i,j,k] >= C[i,j] + C[j,k] - 1)

# Restrições para que p seja o passo máximo do caminho selecionado
@constraint(model, [i = 1:n, j=1:n, k=1:n], p >= P[i,j,k] + (-M * (1 - Q[i,j,k])));

50×50×50 Array{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}, 3}:
[:, :, 1] =
 -9950 Q[1,1,1] + p >= -9950.0   …  -9950 Q[1,50,1] + p >= -9950.0
 -9950 Q[2,1,1] + p >= -3729.0      -9950 Q[2,50,1] + p >= -9950.0
 -9950 Q[3,1,1] + p >= -9950.0      -9950 Q[3,50,1] + p >= -9950.0
 -9950 Q[4,1,1] + p >= -9950.0      -9950 Q[4,50,1] + p >= -9950.0
 -9950 Q[5,1,1] + p >= -9950.0      -9950 Q[5,50,1] + p >= -9950.0
 -9950 Q[6,1,1] + p >= -3504.0   …  -9950 Q[6,50,1] + p >= -9950.0
 -9950 Q[7,1,1] + p >= -9950.0      -9950 Q[7,50,1] + p >= -9950.0
 -9950 Q[8,1,1] + p >= -9950.0      -9950 Q[8,50,1] + p >= -4000.0
 -9950 Q[9,1,1] + p >= -9950.0      -9950 Q[9,50,1] + p >= -9950.0
 -9950 Q[10,1,1] + p >= -9950.0     -9950 Q[10,50,1] + p >= -9950.0
 -9950 Q[11,1,1] + p >= -9950.0  …  -9950 Q[11,50,1] + p >= -2842.0
 -9950 Q[12,1,1] + p >= -9950.0     -9950 Q[12,50,1] + p >= -9950.0
 -9950

# Optimizing

In [ ]:
optimize!(model)
print(objective_value(model))

In [ ]:
print(objective_value(model))